
[](images/wsu_branding.jpg) 
<h1 align="center">
<img src="images/wsu_branding.jpg?raw=1" width="750" height="500" />
</h1>  

<!--Russell Jarvis, Postdoctoral researcher [email](r.jarvis@westernsyndney.edu.au)

-->
The code for this material lives [here](https://github.com/fun-zoological-computing/LearningGitGitHub/blob/master/Week2.ipynb), feel free to hack it.


# COMP 6001 Neuromorphic Algorithms and Computation 

# Week 3 - Tutorial 2 – Optimisation Algorithms and Genetic Algorithms 

 



* Elitism versus NSGA 

* Genes as bitstrings 

* Elitism select best bubble sort 

* Mutation and cross over as operations on bitstrings  

* Pythons GA DEAP module. Override DEAPS MU and Crossover functions. 

* Time varying $ \eta $ (mutation step size). 

Picture of bitstring

In [1]:
import warnings
warnings.filterwarnings('ignore')
!pip install deap

# In Genetic Algorithms an "error function" is similar to a loss function in regular DEAP learning.

In [2]:
import random
import logging
import deap.algorithms
import deap.tools
import pickle
logger = logging.getLogger('__main__')

In [3]:
from deap import benchmarks
benchmarks.zdt1


<function deap.benchmarks.zdt1(individual)>

In [4]:



def _evaluate_invalid_fitness(toolbox, population):
    '''Evaluate the individuals with an invalid fitness
    Returns the count of individuals with invalid fitness
    '''
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    return len(invalid_ind)


def _update_history_and_hof(halloffame, history, population):
    '''Update the hall of fame with the generated individuals
    Note: History and Hall-of-Fame behave like dictionaries
    '''
    if halloffame is not None:
        halloffame.update(population)

    history.update(population)


def _record_stats(stats, logbook, gen, population, invalid_count):
    '''Update the statistics with the new population'''
    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=gen, nevals=invalid_count, **record)


def _get_offspring(parents, toolbox, cxpb, mutpb):
    '''return the offsprint, use toolbox.variate if possible'''
    if hasattr(toolbox, 'variate'):
        return toolbox.variate(parents, toolbox, cxpb, mutpb)
    return deap.algorithms.varAnd(parents, toolbox, cxpb, mutpb)


def eaAlphaMuPlusLambdaCheckpoint(
        population,
        toolbox,
        mu,
        cxpb,
        mutpb,
        ngen,
        stats=None,
        halloffame=None,
        cp_frequency=1,
        cp_filename=None,
        continue_cp=False):
    r"""This is the :math:`(~\alpha,\mu~,~\lambda)` evolutionary algorithm
    Args:
        population(list of deap Individuals)
        toolbox(deap Toolbox)
        mu(int): Total parent population size of EA
        cxpb(float): Crossover probability
        mutpb(float): Mutation probability
        ngen(int): Total number of generation to run
        stats(deap.tools.Statistics): generation of statistics
        halloffame(deap.tools.HallOfFame): hall of fame
        cp_frequency(int): generations between checkpoints
        cp_filename(string): path to checkpoint filename
        continue_cp(bool): whether to continue
    """

    if continue_cp:
        # A file name has been given, then load the data from the file
        cp = pickle.load(open(cp_filename, "r"))
        population = cp["population"]
        parents = cp["parents"]
        start_gen = cp["generation"]
        halloffame = cp["halloffame"]
        logbook = cp["logbook"]
        history = cp["history"]
        random.setstate(cp["rndstate"])
    else:
        # Start a new evolution
        start_gen = 1
        parents = population[:]
        logbook = deap.tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])
        history = deap.tools.History()

        # TODO this first loop should be not be repeated !
        invalid_count = _evaluate_invalid_fitness(toolbox, population)
        _update_history_and_hof(halloffame, history, population)
        _record_stats(stats, logbook, start_gen, population, invalid_count)

    # Begin the generational process
    for gen in range(start_gen + 1, ngen + 1):
        offspring = _get_offspring(parents, toolbox, cxpb, mutpb)

        population = parents + offspring

        invalid_count = _evaluate_invalid_fitness(toolbox, offspring)
        _update_history_and_hof(halloffame, history, population)
        _record_stats(stats, logbook, gen, population, invalid_count)

        # Select the next generation parents
        parents = toolbox.select(population, mu)

        logger.info(logbook.stream)

        if(cp_filename and cp_frequency and
           gen % cp_frequency == 0):
            cp = dict(population=population,
                      generation=gen,
                      parents=parents,
                      halloffame=halloffame,
                      history=history,
                      logbook=logbook,
                      rndstate=random.getstate())
            pickle.dump(cp, open(cp_filename, "wb"))
            logger.debug('Wrote checkpoint to %s', cp_filename)

    return population, logbook, history

In [7]:

import random
import logging
import functools
import numpy

import deap
import deap.base
import deap.algorithms
import deap.tools


class WeightedSumFitness(deap.base.Fitness):

    """Fitness that compares by weighted sum"""

    def __init__(self, values=(), obj_size=None):
        self.weights = [-1.0] * obj_size if obj_size is not None else [-1]

        super(WeightedSumFitness, self).__init__(values)

    @property
    def weighted_sum(self):
        """Weighted sum of wvalues"""
        return sum(self.wvalues)

    @property
    def sum(self):
        """Weighted sum of values"""
        return sum(self.values)

    @property
    def norm(self):
        """Frobenius norm of values"""
        return numpy.linalg.norm(self.values)

    def __le__(self, other):
        return self.weighted_sum <= other.weighted_sum

    def __lt__(self, other):
        return self.weighted_sum < other.weighted_sum

    def __deepcopy__(self, _):
        """Override deepcopy"""

        cls = self.__class__
        result = cls.__new__(cls)
        result.__dict__.update(self.__dict__)
        return result


class WSListIndividual(list):

    """Individual consisting of list with weighted sum field"""

    def __init__(self, *args, **kwargs):
        """Constructor"""
        self.rheobase = None
        self.fitness = WeightedSumFitness(obj_size=kwargs['obj_size'])
        del kwargs['obj_size']
        super(WSListIndividual, self).__init__(*args, **kwargs)


class DEAPOptimisation():

    """DEAP Optimisation class"""

    def get_trans_list(self,param_dict):
        trans_list = []
        for i,k in enumerate([param_dict.keys()]):
            trans_list.append(k)
        return trans_list

    def setnparams(self, nparams = 10, provided_dict = None):
        self.params = optimization_management.create_subset(nparams = nparams,provided_dict = provided_dict)
        self.nparams = len(self.params)
        self.td = self.get_trans_list(self.params)
        return self.params, self.td


    def __init__(self, error_criterion = None, evaluator = None,
                 selection = 'selIBEA',
                 benchmark = False,
                 seed=1,
                 offspring_size=15,
                 elite_size=0,
                 eta=10,
                 mutpb=1.0,
                 cxpb=1.0,
                 map_function=None,
                 backend=None,
                 nparams = 10,
                 provided_dict= {}):
        """Constructor"""

        super(DEAPOptimisation, self).__init__()
        self.selection = selection
        self.benchmark = benchmark
        self.error_criterion = error_criterion
        self.seed = seed
        self.offspring_size = offspring_size
        self.elite_size = elite_size
        self.eta = eta
        self.cxpb = cxpb
        self.mutpb = mutpb
        #self.CPU_NUM = CPU_NUM
        self.backend = backend
        # Create a DEAP toolbox
        self.toolbox = deap.base.Toolbox()
        self.setnparams(nparams = nparams, provided_dict = provided_dict)
        self.setup_deap()



    def set_evaluate(self):
        self.toolbox.register("evaluate", benchmarks.zdt1)
        
    def setup_deap(self):
        """Set up optimisation"""

        # Number of objectives
        #OBJ_SIZE = len(self.evaluator.objectives)

        # Set random seed
        random.seed(self.seed)

        # Eta parameter of crossover / mutation parameters
        # Basically defines how much they 'spread' solution around
        # The lower this value, the more spread
        ETA = self.eta

        # Number of parameters
        # Bounds for the parameters
        IND_SIZE = len(list(self.params.values()))
        OBJ_SIZE = len(self.error_criterion)
        import numpy as np
        LOWER = [ np.min(self.params[v]) for v in self.td ]
        UPPER = [ np.max(self.params[v]) for v in self.td ]
        if self.backend == 'glif':
            for index, i in enumerate(UPPER):
                if i == LOWER[index]:
                    LOWER[index]-=2.0
                    i+=2.0

        # import pdb; pdb.set_trace()
        # Define a function that will uniformly pick an individual
        def uniform(lower_list, upper_list, dimensions):
            """Fill array """

            if hasattr(lower_list, '__iter__'):
                return [random.uniform(lower, upper) for lower, upper in
                        zip(lower_list, upper_list)]
            else:
                return [random.uniform(lower_list, upper_list)
                        for _ in range(dimensions)]

        # Register the 'uniform' function
        self.toolbox.register("uniformparams", uniform, LOWER, UPPER, IND_SIZE)

        # Register the individual format
        # An indiviual is create by WSListIndividual and parameters
        # are initially
        # picked by 'uniform'
        self.toolbox.register(
            "Individual",
            deap.tools.initIterate,
            functools.partial(WSListIndividual, obj_size=OBJ_SIZE),
            self.toolbox.uniformparams)

        # Register the population format. It is a list of individuals
        self.toolbox.register(
            "population",
            deap.tools.initRepeat,
            list,
            self.toolbox.Individual)

        # Register the evaluation function for the individuals
        def custom_code(invalid_ind):
            if self.backend is None:
                invalid_pop = list(update_deap_pop(invalid_ind, self.error_criterion, td = self.td))
            else:
                invalid_pop = list(update_deap_pop(invalid_ind, self.error_criterion, td = self.td, backend = self.backend))
            assert len(invalid_pop) != 0
            invalid_dtc = [ i.dtc for i in invalid_pop if hasattr(i,'dtc') ]
            fitnesses = list(map(evaluate, invalid_dtc))
            return fitnesses

        self.toolbox.register("evaluate", custom_code)
        # Register the mate operator
        self.toolbox.register(
            "mate",
            deap.tools.cxSimulatedBinaryBounded,
            eta=ETA,
            low=LOWER,
            up=UPPER)

        # Register the mutation operator
        self.toolbox.register(
            "mutate",
            deap.tools.mutPolynomialBounded,
            eta=ETA,
            low=LOWER,
            up=UPPER,
            indpb=0.5)

        # Register the variate operator
        self.toolbox.register("variate", deap.algorithms.varAnd)
        self.toolbox.register("select", tools.selIBEA)
        '''
        # Register the selector (picks parents from population)
        if self.selection == str('ngsa'):
            self.toolbox.register("select", tools.selNSGA2)
        elif
        self.selection == str('selIBEA'):
        '''
    def run(self,
            max_ngen=25,
            offspring_size=None,
            continue_cp=False,
            cp_filename=None,
            cp_frequency=0):
        """Run optimisation"""
        # Allow run function to override offspring_size
        # TODO probably in the future this should not be an object field anymore
        # keeping for backward compatibility
        if offspring_size is None:
            offspring_size = self.offspring_size

        # Generate the population object
        pop = self.toolbox.population(n=offspring_size)
        hof = deap.tools.HallOfFame(offspring_size)


        stats = deap.tools.Statistics(key=lambda ind: ind.fitness.sum)
        stats.register("avg", numpy.mean)
        stats.register("std", numpy.std)
        stats.register("min", numpy.min)
        stats.register("max", numpy.max)

        pop, log, history, gen_vs_hof = algorithms.eaAlphaMuPlusLambdaCheckpoint(
            pop,
            self.toolbox,
            offspring_size,
            self.cxpb,
            self.mutpb,
            max_ngen,
            stats=stats,
            halloffame=hof,
            nelite=self.elite_size,
            cp_frequency=cp_frequency,
            continue_cp=continue_cp,
            cp_filename=cp_filename,
            selection = self.selection,
            td = self.td)

        # insert the initial HOF value back in.

        td = self.td
        return pop, hof, log, history, td, gen_vs_hof

